### Betfair Historical Data large dataset creation
1. Following `2_betfair_advanced_data_test.ipynb` steps to download historical data
2. Iterating the stream through a list of files to create a dataset containing multiple events and race days


To connect to the Betfair API through betfairlightweight, you must have first set up your API certificates and saved the login details in dictionary format in a file called `api_logins.json` in the project home directory (`/betfair_project`). The following connects and should return `<LoginResource>` if successful.

In [9]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from pathlib import Path, PurePath #To define open and save locations that are cross-compatible between Windows/Linux
from bz2 import BZ2File #To unzip the Betfair data from its downloaded format
import matplotlib.pyplot as plt

project_dir = Path.cwd().parents[1]
logins_dir = project_dir / 'api_logins.json'

with open(logins_dir) as f:
    login_dict =  json.load(f)
    
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

trading.login()

<LoginResource>

To perform the following, you must have purchased data from the [Betfair Historical data service](https://historicdata.betfair.com/#/home). Advanced data for all sports has been offered for free for Jan - May 2020. To understand more about what different packages of data include, look in the data dictionaries folder in this project. The following lists the data that you have purchased on your Betfair account. Since data is purchased by month, that is how it is represented here:

In [10]:
my_data = trading.historic.get_my_data()
for i in my_data:
    print(i)

{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-01-01T00:00:00', 'purchaseItemId': 41610}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-02-01T00:00:00', 'purchaseItemId': 41610}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-03-01T00:00:00', 'purchaseItemId': 41610}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-04-01T00:00:00', 'purchaseItemId': 41610}
{'sport': 'Horse Racing', 'plan': 'Advanced Plan', 'forDate': '2020-05-01T00:00:00', 'purchaseItemId': 41610}
{'sport': 'Horse Racing', 'plan': 'Pro Plan', 'forDate': '2020-04-01T00:00:00', 'purchaseItemId': 41353}
{'sport': 'Horse Racing', 'plan': 'Pro Plan', 'forDate': '2020-05-01T00:00:00', 'purchaseItemId': 41353}


The following selects just the data for from 01/01/20 to 01/03/20 and returns a dictionary of the contents of the data which we go on to use to download specific country/race/market types, and tells us the size of the entire selection:

In [11]:
collection_options = trading.historic.get_collection_options(
    "Horse Racing", "Advanced Plan", 1, 1, 2020, 1, 3, 2020
)

print(collection_options)

basket_size = trading.historic.get_data_size(
    "Horse Racing", "Advanced Plan", 1, 1, 2020, 1, 3, 2020
)
print(basket_size)

{'marketTypesCollection': [{'name': '', 'count': 1998}, {'name': 'ANTEPOST_WIN', 'count': 22}, {'name': 'DAILY_WIN_DIST', 'count': 1}, {'name': 'EACH_WAY', 'count': 1788}, {'name': 'FORECAST', 'count': 438}, {'name': 'MATCH_BET', 'count': 1692}, {'name': 'OTHER_PLACE', 'count': 4746}, {'name': 'PLACE', 'count': 9161}, {'name': 'RACE_WIN_DIST', 'count': 175}, {'name': 'REV_FORECAST', 'count': 784}, {'name': 'WIN', 'count': 11923}, {'name': 'WITHOUT_FAV', 'count': 306}], 'countriesCollection': [{'name': 'AE', 'count': 511}, {'name': 'AU', 'count': 11874}, {'name': 'ES', 'count': 6}, {'name': 'FR', 'count': 549}, {'name': 'GB', 'count': 10230}, {'name': 'HK', 'count': 1}, {'name': 'IE', 'count': 2147}, {'name': 'NZ', 'count': 2016}, {'name': 'SA', 'count': 44}, {'name': 'SG', 'count': 314}, {'name': 'US', 'count': 4038}, {'name': 'ZA', 'count': 1304}], 'fileTypeCollection': [{'name': 'E', 'count': 1998}, {'name': 'M', 'count': 31036}]}
{'totalSizeMB': 5731, 'fileCount': 33034}


To dig into particular dates, countries, race types and market types we use `.get_file_list()`, specifying our filtering parameters. For example, you can see above that within the sub-dictionary for `'marketTypesCollection'` we have `'WIN'`, among others. The following retrieves a list of the event files that match these parameters. We select only data for 01/02/20, for the `'WIN'` market only and in `'GB'`.

In [12]:
file_list = trading.historic.get_file_list(
    "Horse Racing",
    "Advanced Plan",
    from_day=1,
    from_month=2,
    from_year=2020,
    to_day=1,
    to_month=2,
    to_year=2020,
    market_types_collection=["WIN"],
    countries_collection=["GB"],
    file_type_collection=["M"],
)
print(file_list)

['/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130661.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096552.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136214.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133543.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130666.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096559.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136219.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130671.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133548.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096566.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136224.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133553.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130676.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168130245.bz2', '/xds

To downoad a file, `download_file()` is used, where we specify the download location (on Betfair) in `file_path` and the directory to which it is saved in `store_directory`. The filename is kept as is from Betfair. We both download the file and use the command to assign the file's location to the variable `download` so we can use it in subsequent operations. `file_path` below is a list input over which we iterate `.download_file()`.

In [13]:
data_dir = project_dir / 'data' / 'raw' / 'api'

available_files = file_list[:15]

print(available_files)

downloaded_files = [] #list of directories of each download

for file in available_files:
    download = trading.historic.download_file(file_path = file, store_directory = data_dir)
    print(download)
    downloaded_files.append(download)

['/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130661.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096552.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136214.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133543.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130666.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096559.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136219.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130671.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133548.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168096566.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678828/1.168136224.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/2/29678760/1.168133553.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29678694/1.168130676.bz2', '/xds_nfs/edp_processed/ADVANCED/2020/Feb/1/29677422/1.168130245.bz2', '/xds

The downloaded file is .json formatted compressed in a bzip2 (.bz2) folder. The following extracts the file where it is downloaded. It is given no file extension, however it remains in .txt format and is readable in any text reader.

In [14]:
extracted_files = []

for file in downloaded_files:
    zipfile = BZ2File(file) # open the file
    data = zipfile.read() # get the decompressed data
    newfilepath = file.split('.bz2')[0] # removing the extension and saving without a filetype
    open(newfilepath, 'wb').write(data) # write an uncompressed file
    extracted_files.append(newfilepath)
    zipfile.close()
    
print(extracted_files)

['C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168130661', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168096552', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168136214', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168133543', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168130666', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168096559', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168136219', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168130671', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168133548', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168096566', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168136224', 'C:\\Users\\Asus\\OneDrive\\python\\betfair_project\\data\\raw\\api\\1.168133553', 'C:

## Using stream listener to read extracted data

We now have a readable raw text file in Betfair's dictionary format. To interpret the data with python, betfairlightweight includes a stream listener which will interpret the input of historical data in the same way that it does for live data. The below is edited from [here](https://github.com/liampauling/betfair/blob/104ff4cb8734038cb9351e74d16dc7bd018111bc/examples/examplestreaminghistorical.py). 

This finds the chosen variables in the data file and outputs them in a .csv format in output.txt. Note: output.txt is ignored by git and is not uploaded into the repository.

In [15]:
from betfairlightweight import StreamListener
from betfairlightweight.streaming.stream import MarketStream

class HistoricalStream(MarketStream):
    # create custom listener and stream

    def __init__(self, listener):
        super(HistoricalStream, self).__init__(listener)

    def on_process(self, market_books):
        for market_book in market_books:
            for runner in market_book.runners:
                time.append(market_book.publish_time)
                marketid.append(float(market_book.market_id))
                status.append(market_book.status)
                inplay.append(market_book.inplay)
                selectionid.append(runner.selection_id)
                lastpricetraded.append(runner.last_price_traded)
                totalmatched.append(runner.total_matched)
                sp.append(runner.sp.actual_sp)
                adjfactor.append(runner.adjustment_factor)
                runnerstatus.append(runner.status)
                mkttotalmatched.append(market_book.total_matched)

time = []
marketid = []
status = []
inplay = []
selectionid = []
lastpricetraded = []
totalmatched = []
sp = []
adjfactor = []
runnerstatus = []
mkttotalmatched = []
        
datadict = {'Time': time, 'MarketId' : marketid, 'Status' : status, 'Inplay' : inplay, \
    'SelectionId' : selectionid, 'LastPriceTraded' : lastpricetraded, 'TotalMatched' : totalmatched, \
    'Sp' : sp, 'AdjFactor' :  adjfactor, 'RunnerStatus' : runnerstatus, 'MktTotalMatched' : mkttotalmatched}

class HistoricalListener(StreamListener):
    def _add_stream(self, unique_id, stream_type):
        if stream_type == "marketSubscription":
            return HistoricalStream(self)
        
listener = HistoricalListener(max_latency=None)

for file in extracted_files:
    stream = trading.streaming.create_historical_stream(
        directory=file,
        listener=listener,
    )
    stream.start() 
    print(str(file) + " stream completed.")

C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168130661 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168096552 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168136214 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168133543 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168130666 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168096559 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168136219 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168130671 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168133548 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168096566 stream completed.
C:\Users\Asus\OneDrive\python\betfair_project\data\raw\api\1.168136224 stream completed.
C:\Users\Asus\OneDriv

Iterating the Historical Listener through the list of `extracted_files`. For the full day of racing 01/02/20, `output.txt` is 493mb.

Delete all downloaded files:

In [16]:
# for file in downloaded_files:
#     file = Path(file)
#     file.unlink()

# for file in extracted_files:
#     file = Path(file)
#     file.unlink()

We can now view the data:

In [46]:
df = pd.DataFrame(datadict)
df.head()

,Time,MarketId,Status,Inplay,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,RunnerStatus,MktTotalMatched
0,2020-01-31 11:41:45.697,1.168131,OPEN,False,25545675,0.0,0.0,NaN,5.95,ACTIVE,NaN
1,2020-01-31 11:41:45.697,1.168131,OPEN,False,25238892,0.0,0.0,NaN,30.53,ACTIVE,NaN
2,2020-01-31 11:41:45.697,1.168131,OPEN,False,24707172,0.0,0.0,NaN,7.01,ACTIVE,NaN
3,2020-01-31 11:41:45.697,1.168131,OPEN,False,24420205,0.0,0.0,NaN,8.45,ACTIVE,NaN
4,2020-01-31 11:41:45.697,1.168131,OPEN,False,26314798,0.0,0.0,NaN,33.12,ACTIVE,NaN


In [47]:
df.describe() #descriptive statistics for this data.

,MarketId,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,MktTotalMatched
count,1.197853e+06,1.197853e+06,1.197853e+06,1.197853e+06,25350.000000,1.197853e+06,1.182299e+06
mean,1.168128e+00,1.459440e+07,5.295333e+01,6.819270e+03,70.065700,1.299217e+01,4.893904e+04
std,1.353053e-05,7.147032e+06,1.613125e+02,2.612195e+04,197.044469,1.197839e+01,1.004204e+05
min,1.168097e+00,4.200000e+02,0.000000e+00,0.000000e+00,1.380000,1.000000e-01,0.000000e+00
25%,1.168131e+00,9.673438e+06,4.900000e+00,1.235700e+02,5.160000,4.570000e+00,4.828090e+03
50%,1.168131e+00,1.330615e+07,9.400000e+00,8.688700e+02,10.500000,1.059000e+01,1.786682e+04
75%,1.168136e+00,1.914565e+07,2.700000e+01,3.962300e+03,30.540000,1.792000e+01,4.744522e+04
max,1.168136e+00,2.770676e+07,1.000000e+03,6.559295e+05,1000.000000,7.471000e+01,1.250210e+06


In [48]:
df.info

<bound method DataFrame.info of                            Time  MarketId  Status  Inplay  SelectionId  \
0       2020-01-31 11:41:45.697  1.168131    OPEN   False     25545675   
1       2020-01-31 11:41:45.697  1.168131    OPEN   False     25238892   
2       2020-01-31 11:41:45.697  1.168131    OPEN   False     24707172   
3       2020-01-31 11:41:45.697  1.168131    OPEN   False     24420205   
4       2020-01-31 11:41:45.697  1.168131    OPEN   False     26314798   
...                         ...       ...     ...     ...          ...   
1197848 2020-02-01 14:48:20.153  1.168136  CLOSED    True     15729592   
1197849 2020-02-01 14:48:20.153  1.168136  CLOSED    True     10898328   
1197850 2020-02-01 14:48:20.153  1.168136  CLOSED    True     12745473   
1197851 2020-02-01 14:48:20.153  1.168136  CLOSED    True     11295871   
1197852 2020-02-01 14:48:20.153  1.168136  CLOSED    True     12389735   

         LastPriceTraded  TotalMatched      Sp  AdjFactor RunnerStatus  \
0    

### Creating variables for better analysis

To understand the evolution of matches at each price, we can create `TradeSize` by looking at the change in `TotalMatched` grouped by `LastPriceTraded`.

In [49]:
df['TradeSize'] = df.groupby(['MarketId','SelectionId', 'LastPriceTraded'])['TotalMatched'].diff()

In order to make race timeseries data comparable between events, we need an indicator of what stage the market is in. For example, betting behaviour is very different when the market is first created compared to in the 5 minutes before the off, or compared to in play. A logical anchoring point is the moment in which the race starts, at which point BSP is also defined.

`TimeIndex` is created with time-zero defined at the second where `Inplay == True` for the first time for each `MarketId`, counting positively and negatively in either direction **in seconds**. 

In [50]:
%time

# copying df
df_test = df.copy() # test on sample?

# converting to datetime
df_test['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# calculating inplay start for each race (assigning to new columns)
df_test['start_time'] = df_test['Time'].where(df_test['Inplay'] == True).groupby(df_test['MarketId']).transform('min')

# calculating difference between each time point and start time
df_test['time_dif'] = (df_test['Time'] - df_test['start_time']).astype('timedelta64[s]')

df_test = df_test.drop('start_time', 1)

Wall time: 0 ns


In [51]:
# # both volume/price variation indicate something like:
# 0-20 [every hour?]
# 20-40 [every half hour?]
# 40-45 [every 10 mins]
# then every 1min / 30 seconds if still before off
# then every 10 seconds / second if inplay?

In [52]:
df_test = df_test[df_test['TradeSize'] != 0]


### Analysis: profitable hedge prices in play
* Creating a boolean for prices that would have made a profitable hedge


In [53]:
def lay_hedge_stake(bp, bs, lp, c):
    """
    Function to identify 
    """
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp - c)

def lay_hedge_max_lp(bp, c):
    '''
    Given some back price and commission, this returns the maximum lay price for a profitable hedge bet. The lower the lay price, the more profitable the bet.
    '''
    return (bp - 1) * (1 - c) ** 2 + 1

In [54]:
df_inplay = df_test[(df_test['Inplay'] == True) & (df_test['Status'] == "OPEN")].copy()

In [55]:
df_inplay['LayHedgeProfitable'] = df_inplay['Sp'].apply(lambda x: lay_hedge_max_lp(x, 0.05))

df_inplay['LayHedgeProfitable'] = np.where(df_inplay['LastPriceTraded'] < df_inplay['LayHedgeProfitable'], True, False)

df_inplay.reset_index(inplace=True)

In [56]:
df_inplay.head(10)

,index,Time,MarketId,Status,Inplay,SelectionId,LastPriceTraded,TotalMatched,Sp,AdjFactor,RunnerStatus,MktTotalMatched,TradeSize,time_dif,LayHedgeProfitable
0,58830,2020-02-01 12:35:46.691,1.168131,OPEN,True,25545675,50.00,5614.73,55.00,2.38,ACTIVE,421978.25,3.04,1.0,False
1,58831,2020-02-01 12:35:46.691,1.168131,OPEN,True,25238892,2.28,196939.52,2.47,38.46,ACTIVE,421978.25,82557.33,1.0,True
2,58832,2020-02-01 12:35:46.691,1.168131,OPEN,True,24707172,12.00,14671.79,11.00,6.66,ACTIVE,421978.25,782.02,1.0,False
3,58836,2020-02-01 12:35:47.691,1.168131,OPEN,True,25545675,55.00,5617.82,55.00,2.38,ACTIVE,423065.48,16.65,2.0,False
4,58837,2020-02-01 12:35:47.691,1.168131,OPEN,True,25238892,2.22,197819.57,2.47,38.46,ACTIVE,423065.48,113653.80,2.0,True
5,58838,2020-02-01 12:35:47.691,1.168131,OPEN,True,24707172,18.00,14692.04,11.00,6.66,ACTIVE,423065.48,7885.28,2.0,False
6,58839,2020-02-01 12:35:47.691,1.168131,OPEN,True,24420205,15.00,16666.27,12.00,7.14,ACTIVE,423065.48,4048.34,2.0,False
7,58840,2020-02-01 12:35:47.691,1.168131,OPEN,True,26314798,3.20,153749.55,3.55,35.34,ACTIVE,423065.48,26996.96,2.0,True
8,58841,2020-02-01 12:35:47.691,1.168131,OPEN,True,420,12.00,34520.20,8.78,10.00,ACTIVE,423065.48,33535.36,2.0,False
9,58843,2020-02-01 12:35:48.719,1.168131,OPEN,True,25238892,2.10,198223.29,2.47,38.46,ACTIVE,423727.20,NaN,3.0,True


In [61]:
df_inplay['LayHedgeProfitable'].value_counts(normalize=True)

False    0.630213
True     0.369787
Name: LayHedgeProfitable, dtype: float64

In [62]:
df_inplay = df_inplay[df_inplay['TradeSize'] > 100].copy()

In [63]:
df_inplay['LayHedgeProfitable'].value_counts(normalize=True)

False    0.555115
True     0.444885
Name: LayHedgeProfitable, dtype: float64

In [67]:
df_inplay.groupby('time_dif')['LayHedgeProfitable'].value_counts(normalize=True)

time_dif  LayHedgeProfitable
0.0       False                 0.883333
          True                  0.116667
1.0       False                 0.859649
          True                  0.140351
2.0       False                 0.805556
                                  ...   
344.0     False                 0.500000
          True                  0.500000
345.0     True                  1.000000
346.0     False                 1.000000
349.0     True                  1.000000
Name: LayHedgeProfitable, Length: 675, dtype: float64

In [78]:
#Create dataframe with: time_dif, layhedgeprofitable%, frequency of observations


